In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from CMA_obj import CMA_opt
from PEPG_obj import PEPG_opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from SPSA_obj import SPSA_opt
from ADAM_opt import AdamOptimizer
from PSO_obj import PSO_opt
from scipy.interpolate import interp1d
from numpy import asarray
from numpy import savetxt
from NN_utils import *
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import pandas as pd

#   Ceiling analysis of Neural Networks

- The NN class helper functions and training loop functions are defined in NN_utils, 

### Loading datasets
X is the input, Y the output

In [2]:
# MNIST dataset
MNIST_train = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
MNIST_test = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader_MNIST = torch.utils.data.DataLoader(dataset=MNIST_train, batch_size=100, shuffle=True)
test_loader_MNIST = torch.utils.data.DataLoader(dataset=MNIST_test, batch_size=100, shuffle=False)

X_train_MNIST, Y_train_MNIST = next(iter(train_loader_MNIST))
X_test_MNIST, Y_test_MNIST = next(iter(test_loader_MNIST))

In [5]:
Mini_NN = Tiny_convnet()
N_dim = Mini_NN.count_parameters()
print('Number of parameters in the network: ', N_dim)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Mini_NN.parameters(), lr=0.01)

#training the full NN
n_epochs = 10
test_acc = train_pytorch_NN(Mini_NN, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, optimizer)

Number of parameters in the network:  3594
Using cuda device
Tiny_convnet(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (gap): AdaptiveAvgPool2d(output_size=1)
  (fc): Linear(in_features=16, out_features=10, bias=True)
)
Epoch [1/10], Step [100/600], Loss: 2.1832661628723145, Test Accuracy: 21.73%
Epoch [1/10], Step [200/600], Loss: 1.8369823694229126, Test Accuracy: 29.95%
Epoch [1/10], Step [300/600], Loss: 1.6888870000839233, Test Accuracy: 36.56%
Epoch [1/10], Step [400/600], Loss: 1.6639487743377686, Test Accuracy: 37.27%
Epoch [1/10], Step [500/600], Loss: 1.6073194742202759, Test Accuracy: 42.81%
Epoch [1/10], Step [600/600], Loss: 1.6626931428909302, Test Accuracy: 44.08%
Epoch [2/10], Step [100/600], Loss: 1.5409458875656128, Test Accuracy: 46.39%
Epoch [2/10], Step [200/600], Loss: 1.5673304796218872, Test Accuracy: 46.7%
Epoch [2/10], Step [300/600], Loss: 1.5270144939

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc)), y=test_acc, mode='lines', name='Full NN'))
fig.update_layout(template='plotly_white', width=400, height=400,margin=dict(l=20, r=20, t=20, b=20))


In [12]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.num_filter1 = 8
    self.num_filter2 = 16
    self.num_padding = 2
    # input is 28x28
    # padding=2 for same padding
    self.conv1 = nn.Conv2d(1, self.num_filter1, 5, padding=self.num_padding)
    # feature map size is 14*14 by pooling
    # padding=2 for same padding
    self.conv2 = nn.Conv2d(self.num_filter1, self.num_filter2, 5, padding=self.num_padding)
    # feature map size is 7*7 by pooling
    self.fc = nn.Linear(self.num_filter2*7*7, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_filter2*7*7)   # reshape Variable
    x = self.fc(x)
    return F.log_softmax(x)

In [13]:
Mini_NN = Net()
print(Mini_NN)

Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc): Linear(in_features=784, out_features=10, bias=True)
)
